In [341]:
from nilearn import datasets
import pandas as pd

abide = datasets.fetch_abide_pcp(data_dir="./project/data/nilearn",
                                 pipeline="cpac",
                                 quality_checked=True)

abide_pheno = pd.DataFrame(abide.phenotypic)

groups = []
for s in abide_pheno.SITE_ID:
    groups.append(s.decode()) # for some reason the site names are of type 'bytes'

/Users/kompi/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [342]:
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from argparse import ArgumentParser
import numpy as np
from sklearn.decomposition import PCA
import os
import pandas as pd

def prepare_data(abide, output_dir, pipeline = "cpac", quality_checked = True):
    # get dataset
    print("Loading dataset...")

    # make list of filenames
    fmri_filenames = abide.func_preproc

    # load atlas
    multiscale = datasets.fetch_atlas_basc_multiscale_2015()
    atlas_filename = multiscale.scale064

    # initialize masker object
    masker = NiftiLabelsMasker(labels_img=atlas_filename,
                               standardize=True,
                               memory='nilearn_cache',
                               verbose=0)

    correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True,
                                             discard_diagonal=True)

    try: 
        feat_file = os.path.join(output_dir, 'ABIDE_BASC064_features.npz')
        X_features = np.load(feat_file)['a']
        print("Feature file found.")

    except: # if not, extract features
        X_features = [] # To contain upper half of matrix as 1d array
        print("No feature file found. Extracting features...")

        for i,sub in enumerate(fmri_filenames):
            # extract the timeseries from the ROIs in the atlas
            time_series = masker.fit_transform(sub)
            # create a region x region correlation matrix
            correlation_matrix = correlation_measure.fit_transform([time_series])[0]
            # add to our container
            X_features.append(correlation_matrix)
            # keep track of status
            print('finished extracting %s of %s'%(i+1,len(fmri_filenames)))
        # Save features
        np.savez_compressed(os.path.join(output_dir, 'ABIDE_BASC064_features'),
                                         a = X_features)

    print("Running PCA...")
    pca = PCA(0.99).fit(X_features) 
    X_features_pca = pca.transform(X_features)


    # Transform phenotypic data into dataframe
    abide_pheno = pd.DataFrame(abide.phenotypic)

    # Get the target vector
    y_target = abide_pheno['DX_GROUP'].to_numpy()
    y_target = y_target - 1

    return(X_features_pca, y_target)


In [343]:
X, y = prepare_data(abide, "./project/data/output")

Loading dataset...
Feature file found.
Running PCA...


In [344]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [345]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.autograd import Variable




In [346]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [347]:
class abide_dataset(Dataset):

    def __init__(self, data, labels, transform=None):
        self.data = torch.from_numpy(np.expand_dims(data, axis=1)).type(torch.FloatTensor)
        self.labels = torch.from_numpy(labels).type(torch.LongTensor)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, item):

        return self.data[item], self.labels[item]


In [348]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(571, 128) 
        self.layer_2 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, 2) 
        
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        return x
    

In [349]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (layer_1): Linear(in_features=571, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)


In [350]:
n_epochs = 20
log_interval = 100
batch_size: int = 64
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

train_data = abide_dataset(X_train, y_train, transform=None)
test_data = abide_dataset(X_test, y_test, transform=None)

train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

for epoch in range(n_epochs):
    running_loss = 0.
    for i, (inputs, labels) in enumerate(train_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i + 1) % log_interval == 0:
            progress = int(100 * (i + 1) * train_loader.batch_size / len(train_loader.dataset))
            print(f"\rEpoch {epoch + 1} | Progress: {progress}% | loss: {running_loss / log_interval:.3f}", end="")
            running_loss = 0.0

    with torch.no_grad():        
        correct = 0     
        for i, (inputs, labels) in enumerate(test_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
        
            correct += (outputs.squeeze().argmax(1) == labels).sum().item()

    print(f'\t| Accuracy: {100 * correct / len(X_train):.2f}')

	| Accuracy: 21.35
	| Accuracy: 21.84
	| Accuracy: 26.44
	| Accuracy: 28.57
	| Accuracy: 29.23
	| Accuracy: 29.23
	| Accuracy: 29.06
	| Accuracy: 29.23
	| Accuracy: 29.72
	| Accuracy: 29.72
	| Accuracy: 29.56
	| Accuracy: 29.56
	| Accuracy: 29.56
	| Accuracy: 29.39
	| Accuracy: 29.39
	| Accuracy: 29.39
	| Accuracy: 29.39
	| Accuracy: 29.39
	| Accuracy: 29.39
	| Accuracy: 29.23
